In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
%matplotlib inline
plt.style.use('ggplot')

In [7]:
df1 = pd.read_csv('data/model_features.csv')
df1.head()

,Unnamed: 0,category,main_category,currency,backers,year,month,day,active_days,usd_pledged_real,usd_goal_real,state
0,0,Poetry,Publishing,GBP,0,2015,8,11,58,0.0,1533.95,0
1,1,Narrative Film,Film & Video,USD,15,2017,9,2,59,2421.0,30000.00,0
2,2,Narrative Film,Film & Video,USD,3,2013,1,12,44,220.0,45000.00,0
3,3,Music,Music,USD,1,2012,3,17,29,1.0,5000.00,0
4,4,Film & Video,Film & Video,USD,14,2015,7,4,55,1283.0,19500.00,0


In [13]:
df2 = pd.read_csv('data/nlp.csv')
df2 = df2[['name', 'state']]
df2.head

<bound method NDFrame.head of                                                      name  state
0                         The Songs of Adelaide & Abullah      0
1           Greeting From Earth: ZGAC Arts Capsule For ET      0
2                                          Where is Hank?      0
3       ToshiCapital Rekordz Needs Help to Complete Album      0
4       Community Film Project: The Art of Neighborhoo...      0
...                                                   ...    ...
370440  ChknTruk Nationwide Charity Drive 2014 (Canceled)      0
370441                                          The Tribe      0
370442  Walls of Remedy- New lesbian Romantic Comedy f...      0
370443                           BioDefense Education Kit      0
370444                  Nou Renmen Ayiti!  We Love Haiti!      0

[370445 rows x 2 columns]>